In [2]:
# Imports
import numpy as np
import time
from pydrake.all import (
    DiagramBuilder,
    Simulator,
    StartMeshcat,
    RigidTransform,
    RotationMatrix,
    Sphere,
    Box,
    RollPitchYaw,
    InverseKinematics,
    Solve,
    MakeRenderEngineVtk,
    RenderEngineVtkParams,
)
from pydrake.geometry import Rgba
from pydrake.systems.sensors import RgbdSensor, CameraConfig
from manipulation.station import LoadScenario, MakeHardwareStation

print("✓ Imports loaded")

✓ Imports loaded


In [3]:
meshcat = StartMeshcat()
print(f"Meshcat: {meshcat.web_url()}")

INFO:drake:Meshcat listening for connections at http://localhost:7001


Meshcat: http://localhost:7001


In [1]:
# Perception functions

def depth_image_to_point_cloud(depth_image, rgb_image, camera_info):
    """Convert depth image to colored point cloud in camera frame"""
    height, width = depth_image.shape
    fx = camera_info.focal_x()
    fy = camera_info.focal_y()
    cx = camera_info.center_x()
    cy = camera_info.center_y()
    
    # Pixel grid
    u = np.arange(width)
    v = np.arange(height)
    u_grid, v_grid = np.meshgrid(u, v)
    
    # Valid depth mask
    valid = (depth_image > 0.01) & (depth_image < 5.0)
    
    u_valid = u_grid[valid]
    v_valid = v_grid[valid]
    z_valid = depth_image[valid]
    
    # Back-project to 3D (camera frame: X right, Y down, Z forward)
    x = (u_valid - cx) * z_valid / fx
    y = (v_valid - cy) * z_valid / fy
    z = z_valid
    
    points = np.column_stack([x, y, z])
    colors = rgb_image[valid].astype(float) / 255.0
    
    return points, colors


def transform_points_to_world(points, X_CameraToWorld):
    """Transform points from camera frame to world frame"""
    points_world = []
    for pt in points:
        pt_world = X_CameraToWorld @ pt
        points_world.append(pt_world)
    return np.array(points_world)


def segment_red_color(points, colors, red_min=0.5, other_max=0.35):
    """Segment red points using color thresholds"""
    is_red = (
        (colors[:, 0] > red_min) &      # R > 0.5
        (colors[:, 1] < other_max) &    # G < 0.35
        (colors[:, 2] < other_max)      # B < 0.35
    )
    return points[is_red]


def compute_ball_center(points):
    """Compute centroid of point cluster"""
    if len(points) == 0:
        return None
    return np.mean(points, axis=0)


def simulate_rgbd_capture(camera_info, X_WorldToCamera, ball_positions, ball_colors, ball_radius=0.05):
    """Simulate RGB-D camera seeing balls (for testing without real scene graph objects)"""
    width = camera_info.width()
    height = camera_info.height()
    fx = camera_info.focal_x()
    fy = camera_info.focal_y()
    cx = camera_info.center_x()
    cy = camera_info.center_y()
    
    # Initialize images
    rgb_image = np.ones((height, width, 3), dtype=np.uint8) * 50  # Dark background
    depth_image = np.zeros((height, width), dtype=np.float32)
    
    # Transform balls to camera frame
    X_CameraToWorld = X_WorldToCamera.inverse()
    
    for ball_pos, ball_color in zip(ball_positions, ball_colors):
        ball_in_camera = X_CameraToWorld @ ball_pos
        
        if ball_in_camera[2] <= 0:
            continue
        
        u_center = fx * ball_in_camera[0] / ball_in_camera[2] + cx
        v_center = fy * ball_in_camera[1] / ball_in_camera[2] + cy
        radius_px = int(fx * ball_radius / ball_in_camera[2])
        
        for v in range(max(0, int(v_center - radius_px)), min(height, int(v_center + radius_px))):
            for u in range(max(0, int(u_center - radius_px)), min(width, int(u_center + radius_px))):
                du = u - u_center
                dv = v - v_center
                
                if du**2 + dv**2 < radius_px**2:
                    offset_from_center = np.sqrt(du**2 + dv**2) / radius_px
                    if offset_from_center < 1.0:
                        depth_offset = ball_radius * np.sqrt(1 - offset_from_center**2)
                        depth = ball_in_camera[2] - depth_offset
                        rgb_image[v, u] = ball_color
                        depth_image[v, u] = depth
    
    return rgb_image, depth_image


# Manipulation functions

def compute_sphere_grasp(ball_center, ball_radius):
    """Compute top-down grasp pose for sphere"""
    finger_depth = 0.01
    grasp_height = ball_radius + finger_depth
    p_WG = ball_center + np.array([0, 0, grasp_height])
    R_WG = RotationMatrix(RollPitchYaw([np.pi, 0, 0]))
    X_WG = RigidTransform(R_WG, p_WG)
    return X_WG


def normalize_joint_angles(q_start, q_end):
    """Normalize joint angles to take shortest path (prevents 360 spins!)
    
    For each joint, if the difference is > π, adjust by 2π to take shorter path.
    """
    q_end_normalized = q_end.copy()
    for i in range(len(q_start)):
        diff = q_end[i] - q_start[i]
        # If difference is > π, we're going the long way around
        if diff > np.pi:
            q_end_normalized[i] -= 2 * np.pi
        elif diff < -np.pi:
            q_end_normalized[i] += 2 * np.pi
    return q_end_normalized


def solve_ik_position_priority(plant, plant_context, target_position, 
                                 orientation_target=None, pos_tol=0.005):
    """IK solver prioritizing position over strict orientation
    
    Note: Bin collision geometry exists in scene graph, so the robot
    will respect it during execution even if IK doesn't explicitly check.
    We ensure collision-free paths by approaching from above.
    """
    ik = InverseKinematics(plant, plant_context)
    gripper_frame = plant.GetFrameByName("body")
    
    # Position constraint
    ik.AddPositionConstraint(
        gripper_frame, [0, 0, 0], plant.world_frame(),
        target_position - pos_tol, target_position + pos_tol
    )
    
    # Orientation constraint
    if orientation_target is not None:
        if isinstance(orientation_target, RigidTransform):
            orientation_target = orientation_target.rotation()
        ik.AddOrientationConstraint(
            gripper_frame, RotationMatrix(), plant.world_frame(),
            orientation_target, 0.3
        )
    
    # Cost to stay near current configuration (increased to discourage large movements)
    q_nominal = plant.GetPositions(plant_context, plant.GetModelInstanceByName("iiwa"))
    ik.prog().AddQuadraticErrorCost(np.eye(7) * 50.0, q_nominal, ik.q())  # Increased from 5.0
    
    result = Solve(ik.prog())
    if result.is_success():
        return True, result.GetSolution(ik.q())
    return False, None

In [9]:
# Build scene with robot, gripper, TABLE, and 4 CAMERAS (with PHYSICS!)

import tempfile
import os
from pathlib import Path

# Create temporary directory for SDF files
temp_dir = tempfile.mkdtemp()

# Red ball SDF - with proper inertia for physics
red_ball_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="red_ball">
    <link name="ball">
      <inertial>
        <mass>0.1</mass>
        <inertia>
          <ixx>0.0001</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.0001</iyy>
          <iyz>0</iyz>
          <izz>0.0001</izz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry>
          <sphere>
            <radius>0.05</radius>
          </sphere>
        </geometry>
        <material>
          <diffuse>0.9 0.1 0.1 1.0</diffuse>
        </material>
      </visual>
      <collision name="collision">
        <geometry>
          <sphere>
            <radius>0.05</radius>
          </sphere>
        </geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

# Blue ball SDF
blue_ball_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="blue_ball">
    <link name="ball">
      <inertial>
        <mass>0.1</mass>
        <inertia>
          <ixx>0.0001</ixx>
          <ixy>0</ixy>
          <ixz>0</ixz>
          <iyy>0.0001</iyy>
          <iyz>0</iyz>
          <izz>0.0001</izz>
        </inertia>
      </inertial>
      <visual name="visual">
        <geometry>
          <sphere>
            <radius>0.05</radius>
          </sphere>
        </geometry>
        <material>
          <diffuse>0.2 0.4 0.9 1.0</diffuse>
        </material>
      </visual>
      <collision name="collision">
        <geometry>
          <sphere>
            <radius>0.05</radius>
          </sphere>
        </geometry>
      </collision>
    </link>
  </model>
</sdf>
"""

# Write SDF files
red_ball_path = os.path.join(temp_dir, "red_ball.sdf")
blue_ball_path = os.path.join(temp_dir, "blue_ball.sdf")

with open(red_ball_path, 'w') as f:
    f.write(red_ball_sdf)
    
with open(blue_ball_path, 'w') as f:
    f.write(blue_ball_sdf)

# Ball positions - on TABLE surface (table top at z=0, ball radius=0.05)
ball_radius = 0.05
red_pos = np.array([0.0, -0.5, ball_radius])   # On table, in front of robot
blue_pos = np.array([-0.15, -0.55, ball_radius])  # Slightly to the left

# Table SDF path (from assets)
abs_table_sdf_path = f"{Path.cwd()}/assets/table.sdf"

# Robot and gripper scenario WITH TABLE AND FREE-BODY BALLS (physics enabled!)
# KEY: Using default_free_body_pose instead of add_weld for balls
scenario_yaml = f"""
directives:
- add_model:
    name: iiwa
    file: package://drake_models/iiwa_description/sdf/iiwa7_no_collision.sdf
    default_joint_positions:
      iiwa_joint_1: [-1.57]
      iiwa_joint_2: [0.1]
      iiwa_joint_3: [0]
      iiwa_joint_4: [-1.2]
      iiwa_joint_5: [0]
      iiwa_joint_6: [1.6]
      iiwa_joint_7: [0]
- add_weld:
    parent: world
    child: iiwa::iiwa_link_0
- add_model:
    name: wsg
    file: package://manipulation/hydro/schunk_wsg_50_with_tip.sdf
- add_weld:
    parent: iiwa::iiwa_link_7
    child: wsg::body
    X_PC:
      translation: [0, 0, 0.09]
      rotation: !Rpy {{ deg: [90, 0, 90] }}
- add_model:
    name: table
    file: file://{abs_table_sdf_path}
- add_weld:
    parent: world
    child: table::table_link
    X_PC:
      translation: [0.0, 0.0, -0.05]
      rotation: !Rpy {{ deg: [0, 0, -90] }}
- add_model:
    name: red_ball
    file: file://{red_ball_path}
    default_free_body_pose:
      ball:
        translation: [{red_pos[0]}, {red_pos[1]}, {red_pos[2]}]
- add_model:
    name: blue_ball
    file: file://{blue_ball_path}
    default_free_body_pose:
      ball:
        translation: [{blue_pos[0]}, {blue_pos[1]}, {blue_pos[2]}]

model_drivers:
    iiwa: !IiwaDriver
      control_mode: position_only
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {{}}
"""

meshcat.Delete()
scenario = LoadScenario(data=scenario_yaml)
builder = DiagramBuilder()

# Add hardware station
station = MakeHardwareStation(scenario, meshcat=meshcat)
builder.AddSystem(station)

# Get plant and scene graph
plant = station.GetSubsystemByName("plant")
scene_graph = station.GetSubsystemByName("scene_graph")

# Get initial poses BEFORE building diagram (like arc_trajectory_planning.ipynb cell 3)
temp_context = station.CreateDefaultContext()
temp_plant_context = plant.GetMyContextFromRoot(temp_context)

X_WG_initial = plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName("body"))
print(f"Initial gripper pose: {X_WG_initial.translation()}")

# Add VTK renderer to scene graph
renderer_name = "my_renderer"
params = RenderEngineVtkParams()
renderer = MakeRenderEngineVtk(params)
scene_graph.AddRenderer(renderer_name, renderer)

# Camera setup using CameraConfig
width, height = 640, 480

camera_config = CameraConfig()
camera_config.width = width
camera_config.height = height
camera_config.fps = 10.0
camera_config.renderer_name = renderer_name

# Helper function to create camera transform
def make_camera_transform(camera_pos, look_at_pos):
    """Create camera transform that looks from camera_pos to look_at_pos"""
    forward = look_at_pos - camera_pos
    forward = forward / np.linalg.norm(forward)
    world_up = np.array([0, 0, 1])
    right = np.cross(forward, world_up)
    right = right / np.linalg.norm(right)
    down = np.cross(forward, right)
    R_WorldToCamera = RotationMatrix(np.column_stack([right, down, forward]))
    return RigidTransform(R_WorldToCamera, camera_pos)

# Define 4 camera positions SPREAD AROUND the workspace
workspace_center = np.array([0.0, -0.5, 0.05])

camera_1_pos = np.array([0.4, -0.15, 0.5])      # Front-right, HIGH
camera_2_pos = np.array([-0.35, -0.9, 0.25])    # Back-left, LOW
camera_3_pos = np.array([-0.5, -0.4, 0.4])      # Left side, MEDIUM
camera_4_pos = np.array([0.5, -0.6, 0.35])      # RIGHT SIDE

X_WorldToCamera_1 = make_camera_transform(camera_1_pos, workspace_center)
X_WorldToCamera_2 = make_camera_transform(camera_2_pos, workspace_center)
X_WorldToCamera_3 = make_camera_transform(camera_3_pos, workspace_center)
X_WorldToCamera_4 = make_camera_transform(camera_4_pos, workspace_center)

# Create 4 sets of cameras
color_camera_1, depth_camera_1 = camera_config.MakeCameras()
color_camera_2, depth_camera_2 = camera_config.MakeCameras()
color_camera_3, depth_camera_3 = camera_config.MakeCameras()
color_camera_4, depth_camera_4 = camera_config.MakeCameras()

# Add 4 RgbdSensors to diagram
rgbd_sensor_1 = builder.AddSystem(RgbdSensor(
    parent_id=scene_graph.world_frame_id(),
    X_PB=X_WorldToCamera_1,
    color_camera=color_camera_1,
    depth_camera=depth_camera_1
))

rgbd_sensor_2 = builder.AddSystem(RgbdSensor(
    parent_id=scene_graph.world_frame_id(),
    X_PB=X_WorldToCamera_2,
    color_camera=color_camera_2,
    depth_camera=depth_camera_2
))

rgbd_sensor_3 = builder.AddSystem(RgbdSensor(
    parent_id=scene_graph.world_frame_id(),
    X_PB=X_WorldToCamera_3,
    color_camera=color_camera_3,
    depth_camera=depth_camera_3
))

rgbd_sensor_4 = builder.AddSystem(RgbdSensor(
    parent_id=scene_graph.world_frame_id(),
    X_PB=X_WorldToCamera_4,
    color_camera=color_camera_4,
    depth_camera=depth_camera_4
))

# Connect all 4 sensors to scene graph
builder.Connect(station.GetOutputPort("query_object"), rgbd_sensor_1.query_object_input_port())
builder.Connect(station.GetOutputPort("query_object"), rgbd_sensor_2.query_object_input_port())
builder.Connect(station.GetOutputPort("query_object"), rgbd_sensor_3.query_object_input_port())
builder.Connect(station.GetOutputPort("query_object"), rgbd_sensor_4.query_object_input_port())

# Build diagram and publish to meshcat
diagram = builder.Build()
context = diagram.CreateDefaultContext()
diagram.ForcedPublish(context)

# Home position
q_home = np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0])

# Visualize all 4 cameras in meshcat
from pydrake.geometry import Cylinder

camera_colors = [
    Rgba(0.0, 0.5, 1.0, 0.8),   # Cyan
    Rgba(1.0, 0.5, 0.0, 0.8),   # Orange
    Rgba(0.8, 0.0, 0.8, 0.8),   # Purple
    Rgba(0.0, 1.0, 0.5, 0.8),   # Teal
]

camera_transforms = [X_WorldToCamera_1, X_WorldToCamera_2, X_WorldToCamera_3, X_WorldToCamera_4]
camera_positions = [camera_1_pos, camera_2_pos, camera_3_pos, camera_4_pos]

for i, (X_cam, pos, color) in enumerate(zip(camera_transforms, camera_positions, camera_colors), 1):
    meshcat.SetObject(f"camera{i}/body", Box(0.05, 0.05, 0.08), color)
    meshcat.SetTransform(f"camera{i}/body", X_cam)
    
    lens_color = Rgba(color.r() * 0.5, color.g() * 0.5, color.b() * 0.5, 0.9)
    meshcat.SetObject(f"camera{i}/lens", Cylinder(0.02, 0.03), lens_color)
    meshcat.SetTransform(f"camera{i}/lens", X_cam @ RigidTransform([0, 0, 0.05]))

# Save camera info and sensors for perception
camera_info_1 = color_camera_1.core().intrinsics()
camera_info_2 = color_camera_2.core().intrinsics()
camera_info_3 = color_camera_3.core().intrinsics()
camera_info_4 = color_camera_4.core().intrinsics()

camera_sensors = [rgbd_sensor_1, rgbd_sensor_2, rgbd_sensor_3, rgbd_sensor_4]
camera_infos = [camera_info_1, camera_info_2, camera_info_3, camera_info_4]
camera_transforms = [X_WorldToCamera_1, X_WorldToCamera_2, X_WorldToCamera_3, X_WorldToCamera_4]

print("✓ PHYSICS-ENABLED SETUP: Robot + Table + FREE-BODY Balls + 4 CAMERAS")
print(f"  Resolution: {width}x{height} per camera")
print(f"  Red ball at: {red_pos} (FREE BODY)")
print(f"  Blue ball at: {blue_pos} (FREE BODY)")
print(f"\n  Cameras: Cyan (front-right), Orange (back-left), Purple (left), Teal (right)")
print(f"\n✓ Ready for perception (cell 5), then physics simulation (cell 6)")

Initial gripper pose: [ 3.70783219e-04 -4.65616808e-01  6.79321579e-01]
✓ PHYSICS-ENABLED SETUP: Robot + Table + FREE-BODY Balls + 4 CAMERAS
  Resolution: 640x480 per camera
  Red ball at: [ 0.   -0.5   0.05] (FREE BODY)
  Blue ball at: [-0.15 -0.55  0.05] (FREE BODY)

  Cameras: Cyan (front-right), Orange (back-left), Purple (left), Teal (right)

✓ Ready for perception (cell 5), then physics simulation (cell 6)


In [ ]:
# Visualization helpers (no bin in throwing pipeline)
meshcat.SetObject("markers/red_target", Sphere(0.01), Rgba(1, 0.5, 0, 0.8))
meshcat.SetTransform("markers/red_target", RigidTransform(red_pos))

meshcat.SetObject("markers/blue_target", Sphere(0.01), Rgba(0, 0.5, 1, 0.8))
meshcat.SetTransform("markers/blue_target", RigidTransform(blue_pos))

print("✓ Scene ready for perception")
print(f"  Red ball ground truth: {red_pos}")
print(f"  Blue ball ground truth: {blue_pos}")

✓ Scene ready for perception
  Red ball ground truth: [ 0.   -0.5   0.05]
  Blue ball ground truth: [-0.15 -0.55  0.05]


In [11]:
# PERCEPTION: Multi-camera capture and point cloud fusion (4 CAMERAS)

print("=== MULTI-CAMERA PERCEPTION PIPELINE ===\n")

# Lists to store point clouds from all cameras
all_points_world = []
all_colors = []

# Process each camera
for cam_idx, (sensor, cam_info, X_cam) in enumerate(zip(camera_sensors, camera_infos, camera_transforms), 1):
    print(f"--- Camera {cam_idx} ---")
    
    # Get images from this sensor
    sensor_context = sensor.GetMyContextFromRoot(context)
    
    color_image = sensor.color_image_output_port().Eval(sensor_context)
    rgb_img = color_image.data[:, :, :3].astype(np.uint8)
    
    depth_image = sensor.depth_image_32F_output_port().Eval(sensor_context)
    depth_img = depth_image.data.squeeze().astype(np.float32)
    
    # Convert to point cloud in camera frame
    points_camera, colors = depth_image_to_point_cloud(depth_img, rgb_img, cam_info)
    print(f"  Generated {len(points_camera)} points")
    
    # Transform to world frame
    points_world = transform_points_to_world(points_camera, X_cam)
    
    all_points_world.append(points_world)
    all_colors.append(colors)

# Merge all point clouds
merged_points = np.vstack(all_points_world)
merged_colors = np.vstack(all_colors)

print(f"\n✓ Merged {len(merged_points)} points from {len(camera_sensors)} cameras\n")

# VISUALIZE point clouds from each camera with DIFFERENT COLORS
print("--- Visualizing Point Clouds (color-coded by camera) ---")
cam_viz_colors = [
    Rgba(0.0, 0.5, 1.0, 0.9),   # Cyan - Camera 1
    Rgba(1.0, 0.5, 0.0, 0.9),   # Orange - Camera 2  
    Rgba(0.8, 0.0, 0.8, 0.9),   # Purple - Camera 3
    Rgba(0.0, 1.0, 0.5, 0.9),   # Teal - Camera 4
]
cam_names = ['Cyan', 'Orange', 'Purple', 'Teal']

# Downsample for visualization
downsample = 300

for cam_idx, (points, viz_color) in enumerate(zip(all_points_world, cam_viz_colors)):
    print(f"  Camera {cam_idx+1}: plotting {len(points)//downsample} points ({cam_names[cam_idx]})")
    for i in range(0, len(points), downsample):
        meshcat.SetObject(f"pointcloud/cam{cam_idx+1}/pt_{i}", Sphere(0.004), viz_color)
        meshcat.SetTransform(f"pointcloud/cam{cam_idx+1}/pt_{i}", RigidTransform(points[i]))

print(f"  ✓ Plotted ~{len(merged_points)//downsample} points total")
print(f"  Legend: Cyan=Cam1, Orange=Cam2, Purple=Cam3, Teal=Cam4\n")

# BALL DETECTION
print("--- Ball Detection ---")

# Step 1: Find points that are ABOVE the table (z > 0.02)
above_table_mask = merged_points[:, 2] > 0.02
above_table_points = merged_points[above_table_mask]
above_table_colors = merged_colors[above_table_mask]
print(f"Points above table (z > 0.02): {len(above_table_points)}")

# Step 2: Among elevated points, find the reddest ones
if len(above_table_points) > 0:
    redness = above_table_colors[:, 0] - np.maximum(above_table_colors[:, 1], above_table_colors[:, 2])
    
    num_red_points = max(100, int(len(above_table_points) * 0.05))
    red_threshold = np.sort(redness)[-num_red_points]
    
    is_red = redness >= red_threshold
    red_points = above_table_points[is_red]
    print(f"Reddest {num_red_points} elevated points selected")
    
    # Step 3: Cluster to find ball center
    if len(red_points) > 10:
        centroid = np.mean(red_points, axis=0)
        
        for iteration in range(3):
            distances = np.linalg.norm(red_points - centroid, axis=1)
            inliers = distances < 0.06
            if np.sum(inliers) > 10:
                centroid = np.mean(red_points[inliers], axis=0)
                red_points = red_points[inliers]
        
        print(f"After clustering: {len(red_points)} points")
        
        detected_red_pos = np.mean(red_points, axis=0)
        detected_red_pos[2] = ball_radius
        
        error = np.linalg.norm(detected_red_pos - red_pos)
        print(f"\nDetected: {detected_red_pos}")
        print(f"Ground truth: {red_pos}")
        print(f"Error: {error*1000:.1f}mm")
        
        # Visualize detected ball (bright yellow)
        meshcat.SetObject("perception/detected", Sphere(0.055), Rgba(1.0, 1.0, 0.0, 0.6))
        meshcat.SetTransform("perception/detected", RigidTransform(detected_red_pos))
        
        # Visualize clustered red points (bright green for visibility)
        for i in range(0, len(red_points), max(1, len(red_points)//30)):
            meshcat.SetObject(f"perception/red_pt_{i}", Sphere(0.008), Rgba(0, 1, 0, 1.0))
            meshcat.SetTransform(f"perception/red_pt_{i}", RigidTransform(red_points[i]))
        
        print("\n" + "="*60)
        print(f"✓ DETECTED RED BALL - Error: {error*1000:.1f}mm")
        print("="*60)
    else:
        detected_red_pos = None
        print("✗ Too few red points for clustering")
else:
    detected_red_pos = None
    print("✗ No elevated points found")

=== MULTI-CAMERA PERCEPTION PIPELINE ===

--- Camera 1 ---
  Generated 266226 points
--- Camera 2 ---
  Generated 246300 points
--- Camera 3 ---
  Generated 270377 points
--- Camera 4 ---
  Generated 270942 points

✓ Merged 1053845 points from 4 cameras

--- Visualizing Point Clouds (color-coded by camera) ---
  Camera 1: plotting 887 points (Cyan)
  Camera 2: plotting 821 points (Orange)
  Camera 3: plotting 901 points (Purple)
  Camera 4: plotting 903 points (Teal)
  ✓ Plotted ~3512 points total
  Legend: Cyan=Cam1, Orange=Cam2, Purple=Cam3, Teal=Cam4

--- Ball Detection ---
Points above table (z > 0.02): 83671
Reddest 4183 elevated points selected
After clustering: 4244 points

Detected: [-0.00219639 -0.50580673  0.05      ]
Ground truth: [ 0.   -0.5   0.05]
Error: 6.2mm

✓ DETECTED RED BALL - Error: 6.2mm


In [12]:
# MANIPULATION: Pick the detected red ball - WITH PHYSICS (like arc_trajectory_planning.ipynb)

from pydrake.all import PiecewisePose, PiecewisePolynomial, TrajectorySource

if detected_red_pos is None:
    print("ERROR: No ball detected, cannot proceed with manipulation")
else:
    print("=== MANIPULATION: Pick Detected Ball (Physics Simulation) ===\n")
    
    # Import helper from throw_helpers
    import sys
    sys.path.append(str(Path.cwd()))
    from throw_helpers import create_q_knots, joint_angles_to_pose
    
    target_pos = detected_red_pos
    print(f"Target ball position: {target_pos}\n")
    
    # Get initial gripper pose (from cell 3's temp_context)
    print(f"Initial gripper pose: {X_WG_initial.translation()}")
    
    # Ball pose (as RigidTransform)
    X_WBall = RigidTransform(p=target_pos)
    
    # Design grasp pose: ABOVE the ball, gripper pointing DOWN
    GRASP_HEIGHT = 0.12
    APPROACH_HEIGHT = 0.1
    GRIPPER_OPEN = 0.107
    GRIPPER_CLOSED = 0.0
    
    def design_grasp_pose(X_WO: RigidTransform) -> RigidTransform:
        """Grasp pose above object with gripper pointing down."""
        return X_WO @ RigidTransform(RotationMatrix.MakeXRotation(-np.pi/2), [0, 0, GRASP_HEIGHT])
    
    X_WG_grasp = design_grasp_pose(X_WBall)
    X_WG_approach = RigidTransform(p=[0, 0, APPROACH_HEIGHT]) @ X_WG_grasp
    
    print(f"Grasp pose: {X_WG_grasp.translation()}")
    print(f"Approach pose: {X_WG_approach.translation()}")
    
    # Trajectory timing
    t_approach = 1.0
    t_descend = 0.5
    t_grasp = 0.5   # Hold time for gripper to close
    t_lift = 0.5
    
    t0 = 0
    t1 = t_approach
    t2 = t1 + t_descend
    t3 = t2 + t_grasp
    t4 = t3 + t_lift
    
    print(f"\nTrajectory: {t4:.1f}s total")
    print(f"  t={t0:.1f}s: Initial")
    print(f"  t={t1:.1f}s: Approach")
    print(f"  t={t2:.1f}s: Grasp pose")
    print(f"  t={t3:.1f}s: Hold complete (gripper closes)")
    print(f"  t={t4:.1f}s: Lift")
    
    # Create pose trajectory
    pose_traj = PiecewisePose.MakeLinear(
        [t0, t1, t2, t3, t4],
        [X_WG_initial, X_WG_approach, X_WG_grasp, X_WG_grasp, X_WG_approach]
    )
    
    # Gripper trajectory: open until t2, then close
    gripper_traj = PiecewisePolynomial.FirstOrderHold(
        [t0, t2, t2 + 0.01, t4],
        np.array([[GRIPPER_OPEN, GRIPPER_OPEN, GRIPPER_CLOSED, GRIPPER_CLOSED]])
    )
    
    # Convert poses to joint angles via IK
    print("\nSolving IK for trajectory...")
    num_samples = 50
    t_samples = np.linspace(t0, t4, num_samples)
    poses = [pose_traj.GetPose(t) for t in t_samples]
    q_knots = create_q_knots(plant, poses)
    
    print(f"✓ IK solved for {len(q_knots)}/{num_samples} poses")
    
    if len(q_knots) >= 10:
        # Create joint trajectory
        q_traj = PiecewisePolynomial.CubicShapePreserving(
            t_samples[:len(q_knots)], 
            np.array(q_knots).T
        )
        
        print(f"\n=== Building Physics Simulation ===")
        
        # Rebuild diagram with trajectory sources (like arc_trajectory_planning cell 7)
        meshcat.Delete()
        builder2 = DiagramBuilder()
        scenario2 = LoadScenario(data=scenario_yaml)
        station2 = MakeHardwareStation(scenario2, meshcat=meshcat)
        builder2.AddSystem(station2)
        
        # Connect trajectory sources to station inputs
        q_source = builder2.AddSystem(TrajectorySource(q_traj))
        g_source = builder2.AddSystem(TrajectorySource(gripper_traj))
        
        builder2.Connect(q_source.get_output_port(), station2.GetInputPort("iiwa.position"))
        builder2.Connect(g_source.get_output_port(), station2.GetInputPort("wsg.position"))
        
        diagram2 = builder2.Build()
        
        # Run physics simulation
        simulator2 = Simulator(diagram2)
        simulator2.set_target_realtime_rate(1.0)
        
        print(f"Running physics simulation...")
        print(f"  Gripper closes at t={t2:.2f}s")
        
        meshcat.StartRecording()
        simulator2.AdvanceTo(q_traj.end_time() + 1.0)
        meshcat.StopRecording()
        meshcat.PublishRecording()
        
        print("\n" + "="*60)
        print("✓ PHYSICS SIMULATION COMPLETE!")
        print(f"  Total time: {q_traj.end_time() + 1.0:.2f}s")
        print("  Ball should respond to gripper contact")
        print("="*60)
    else:
        print("✗ Not enough IK solutions - check robot reachability")

=== MANIPULATION: Pick Detected Ball (Physics Simulation) ===

Target ball position: [-0.00219639 -0.50580673  0.05      ]

Initial gripper pose: [ 3.70783219e-04 -4.65616808e-01  6.79321579e-01]
Grasp pose: [-0.00219639 -0.50580673  0.17      ]
Approach pose: [-0.00219639 -0.50580673  0.27      ]

Trajectory: 2.5s total
  t=0.0s: Initial
  t=1.0s: Approach
  t=1.5s: Grasp pose
  t=2.0s: Hold complete (gripper closes)
  t=2.5s: Lift

Solving IK for trajectory...
✓ IK solved for 50/50 poses

=== Building Physics Simulation ===
Running physics simulation...
  Gripper closes at t=1.50s

✓ PHYSICS SIMULATION COMPLETE!
  Total time: 3.50s
  Ball should respond to gripper contact
